In [ ]:
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D
from keras.models import Model
from keras.optimizers import RMSprop

In [ ]:
def noisy(image):
    row,col= image.shape
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    noisy = image + gauss
    return noisy

In [ ]:
def s_p(image):
    row,col = image.shape
    s_vs_p = 0.5
    amount = 0.05
    out = np.copy(image)
      # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    out[coords] = 1

      # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    out[coords] = 0
    return out

In [ ]:

def load_image(path):
    list_file = os.listdir(path)
    random.seed(40)
    random.shuffle(list_file)
    x_data = []
    y_data = []
    for file in list_file:
        im = cv2.resize(cv2.imread(path + file,  cv2.IMREAD_GRAYSCALE), (224, 224))
        im = img_to_array(im)
        x_data.append(im)
    x_data = np.array(x_data, dtype='float')/255.0
    y_data = np.array(noisy(x_data))
    return x_data, y_data

In [ ]:
def autoencoder():
    #encoder
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(224, 224, 1)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation("relu"))

    #decoder
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation("relu"))
    model.add(UpSampling2D((2,2)))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation("relu"))
    model.add(UpSampling2D((2,2)))
    model.add(Conv2D(1, (3, 3), padding='same'))
    model.add(Activation("sigmoid"))
    return model

In [ ]:
IMG_PATH_TRAIN = '../Imagetrain/train/'
IMG_PATH_TEST = '../Imagetrain/test/'
model = autoencoder()
model.compile(loss='mean_squared_error', optimizer = RMSprop())
X_train, X_train_noise = load_image(IMG_PATH_TRAIN)
X_test, X_test_noise = load_image(IMG_PATH_TEST)
log = callbacks.CSVLogger('result/log.csv')
tb = callbacks.TensorBoard(log_dir='result/tensorboard-logs',
                         batch_size=128)
checkpoint = callbacks.ModelCheckpoint('result/weights-{epoch:02d}.h5', monitor='val_acc',
                                           save_best_only=True, save_weights_only=True, verbose=1)
history = model.fit(X_train, X_train_noise_train, batch_size=128, epochs=20, verbose=1,
                        validation_data=(X_test, X_test_noise),
                    validation_split=0.25, callbacks=[log, tb, checkpoint])
score = model.evaluate(X_test, Y_test, verbose=1)
print('Test score:', score[0])
print('Test accuracy', score[1])